In [14]:
# %pip install numpy
# %pip install pandas
# %pip install gspread
# %pip install dotmap

In [36]:
# IMPORTS
import pandas as pd
import numpy as np
import gspread
from dotmap import DotMap
import math
import os, sys
from collections import deque 

# Silence game state printing
SILENT = False
def toggle_silent():
    global SILENT
    SILENT = not SILENT

def enable_silent():
    global SILENT
    SILENT = True

def disable_silent():
    global SILENT
    SILENT = False

# Fully disable/enable print
PRINT = True
def print(*args, **kwargs):
    if PRINT:
        __builtins__.print(*args, **kwargs)

def disable_print():
    global PRINT
    PRINT = False

def enable_print():
    global PRINT
    PRINT = True

In [16]:
# Google Sheets API
sa = gspread.service_account(filename="./../gd-client-key.json")
sheet = sa.open("Goblins and Glory Params")

game = DotMap()
items_df = pd.DataFrame()
hero_df = pd.DataFrame()


In [68]:
# DATA and PARAMETERS
MAX_TIME = 120
WIN_GOLD = 1000
EXPLORE_TIME = 1
HIT_TIME = 1
MOVE_TIME = 2

INIT_MAP = [
    ['G2', 'T2', 'E1', 'T1', 'E0', '00'],
    ['T1', 'E0', 'G1', 'E1', 'G0', 'T0'],
    ['E1', 'E0', 'E2', 'E2', 'E0', 'E0'],
    ['T0', 'G0', 'E1', 'G1', 'E0', 'T1'],
    ['00', 'E0', 'T1', 'E1', 'T2', 'G2'],
]

LEGEND = {
    'T': 'treasure',
    'E': 'monster',
    'G': 'statue'
}

def update_data():
    global items_df, hero_df
    hero_df = pd.DataFrame(sheet.get_worksheet(0).get_all_records())
    items_df = pd.DataFrame(sheet.get_worksheet(1).get_all_records())

update_data()
print("Hero \n", hero_df.head())
print("Items \n", items_df)

Hero 
    level  health  attackDamage  gloryForLevel
0      0     100            10            100
1      1     125            20            300
2      2     150            30            600
3      3     175            40           1000
4      4     200            50           1500
Items 
        type  tier  glory  gold  maxHealth  attackDamage  cooldown  \
0   monster     0     25    25         20            20        10   
1   monster     1     40    40         45            35        15   
2   monster     2     60    60        100            50        20   
3  treasure     0      0    50          0             0         5   
4  treasure     1      0    75          0             0        10   
5  treasure     2      0   100          0             0        15   
6    statue     0     50     0          0             0         5   
7    statue     1    100     0          0             0        10   
8    statue     2    150     0          0             0        15   

   timeToConsume0 

In [106]:
# Game functions

def create_hero():
    hero = DotMap()
    hero.health = hero_df['health'][0]
    hero.attack = hero_df['attackDamage'][0]
    hero.level = 0
    hero.glory = 0
    hero.gold = 0
    hero.name = "Hero"
    SILENT or print("Hero created")
    return hero 

def damage_thang(thang, damage):
    thang.health -= damage
    if thang.health <= 0:
        thang.health = 0
        SILENT or print(f'{thang.name} died!')
        return False
    return True

def add_glory(hero, glory):
    hero.glory += glory
    glory_for_next_level = hero_df['gloryForLevel'].get(hero.level, math.inf)
    SILENT or print("Hero gained {} glory, has {}, needs {} glory for level {}".format(glory, hero.glory, glory_for_next_level, hero.level + 1))
    if hero.glory >= glory_for_next_level:
        add_level(hero)

def add_gold(hero, gold):
    hero.gold += gold
    SILENT or print("Hero gained {} gold, has {}".format(gold, hero.gold))

def add_level(hero):
    hero.level += 1
    SILENT or print("Hero gained a level!")
    hero.health = hero_df['health'][hero.level]
    hero.attack = hero_df['attackDamage'][hero.level]

def create_monster(tier):
    monster = DotMap()
    m_data = items_df[(items_df['tier'] == tier) & (items_df['type'] == 'monster')]
    if m_data.empty:
        SILENT or print("No monster for tier {}".format(tier))
        return None
    monster.health = m_data['maxHealth'].iloc[0]
    monster.maxHealth = m_data['maxHealth'].iloc[0]
    monster.attack = m_data['attackDamage'].iloc[0]
    return monster

def create_statue(tier):
    statue = DotMap()
    s_data = items_df[(items_df['tier'] == tier) & (items_df['type'] == 'statue')]
    if s_data.empty:
        SILENT or print("No statue for tier {}".format(tier))
        return None
    return statue

def create_treasure(tier):
    treasure = DotMap()
    t_data = items_df[(items_df['tier'] == tier) & (items_df['type'] == 'treasure')]
    if t_data.empty:
        SILENT or print("No treasure for tier {}".format(tier))
        return None
    
    return treasure

def create_item(type, tier):
    item = None
    if type == 'monster':
        item = create_monster(tier)
    elif type == 'statue':
        item = create_statue(tier)
    elif type == 'treasure':
        item = create_treasure(tier)
    else:
        SILENT or print("Unknown item type {}".format(type))
        return None
    i_data = items_df[(items_df['tier'] == tier) & (items_df['type'] == type)]
    item.gold = i_data['gold'].iloc[0]
    item.glory = i_data['glory'].iloc[0]
    item.name = "{}{}".format(type, tier)
    item.type = type
    item.cooldown = i_data['cooldown'].iloc[0]
    return item

def open_treasure(hero, treasure):
    SILENT or print("Hero opens treasure")
    add_gold(hero, treasure.gold)
    return True

def pray_statue(hero, statue):
    SILENT or print("Hero prays to statue")
    add_glory(hero, statue.glory)
    return True

def fight_monster(hero, monster):
    SILENT or print("Hero fights monster")
    while True:
        if not damage_thang(hero, monster.attack):
            return False
        if not damage_thang(monster, hero.attack):
            add_glory(hero, monster.glory)
            add_gold(hero, monster.gold)
            return True

def create_map(map_pattern):
    map = []
    for row in map_pattern:
        line = []
        for lt in row:
            if lt == '00':
                line.append(None)
            else:
                line.append(create_item(LEGEND[lt[0]], int(lt[1])))
        map.append(line)
    return map

def create_game(game_map):
    game = DotMap()
    game.hero = create_hero()
    game.map = create_map(game_map)
    game.rows = len(game.map)
    game.cols = len(game.map[0])
    game.hrow = game.rows - 1
    game.hcol = 0
    game.game_over = False
    game.win_gold = WIN_GOLD
    game.max_time = MAX_TIME
    game.time = 0
    game.won = False
    SILENT or print("Game created")
    return game

def move_hero(game, direction):
    if game.game_over:
        SILENT or print("Game is over")
        return False
    prow = game.hrow
    pcol = game.hcol
    if direction == 'up':
        if game.hrow > 0:
            game.hrow -= 1
    elif direction == 'down':
        if game.hrow < game.rows - 1:
            game.hrow += 1
    elif direction == 'left':
        if game.hcol > 0:
            game.hcol -= 1
    elif direction == 'right':
        if game.hcol < game.cols - 1:
            game.hcol += 1
    else:
        print("Unknown direction {}".format(direction))
        return False
    if game.hrow == prow and game.hcol == pcol:
        SILENT or print("Hero cannot move in that direction")
        return False
    else:
        game.time += MOVE_TIME
        SILENT or print(f"Hero moved from [{prow}, {pcol}] to [{game.hrow}, {game.hcol}]")
        check_game(game)
        return True

def get_item(game):
    item = game.map[game.hrow][game.hcol]
    if item is None or item.respawnAt:
        SILENT or print("No item to get")
        return None
    return item

def explore(game):
    if game.hero.health <= 0:
        return False
    item = get_item(game)
    game.time += 1
    if item is None:
        SILENT or print("Nothing to explore")
        return False
    result = False
    if item.type == 'monster':
        game.time += math.ceil(item.health / game.hero.attack)
        result = fight_monster(game.hero, item)
    elif item.type == 'statue':
        result = pray_statue(game.hero, item)
    elif item.type == 'treasure':
        result = open_treasure(game.hero, item)
    else:
        SILENT or print("Unknown item type {}".format(item.type))
    if not result:
        SILENT or print("Game over")
        game.game_over = True
    item.respawnAt = game.time + item.cooldown
    check_game(game)
    return result

def check_game(game):
    for row in game.map:
        for item in row:
            if item and item.respawnAt and item.respawnAt <= game.time:
                item.respawnAt = None
                if item.type == 'monster':
                    item.health = item.maxHealth
    if game.hero.gold >= game.win_gold:
        SILENT or print("Hero won")
        game.won = True
        game.game_over = True
    if game.time >= game.max_time:
        SILENT or print("Game over")
        game.game_over = True

def print_game(game, withoutMap=False):
    hero = game.hero
    print(f"Hero [{game.hrow, game.hcol}] : {hero.level} Level, {hero.health} HP, {hero.gold} gold, {hero.glory} glory")
    print(f"Time: {game.time}. Hero {'alive' if hero.health > 0 else 'dead'}. Game {'in progress' if game.game_over else 'over'}. Game {'won' if game.won else 'lost'}")
    if withoutMap:
        return
    for row in game.map:
        for item in row:
            if item is None:
                print("  , ", end='')
            elif item.respawnAt:
                print(f"{item.name[0]}{item.name[-1]}, ", end='')
            else:
                print(f"{item.name[0].capitalize()}{item.name[-1]}, ", end='')
        print()



In [102]:
import random

# S2, T2, M1, T1, M0,   , 
# T1, M0, S1, M1, S0, T0, 
# M1, M0, M2, M2, M0, M0, 
# T0, S0, M1, S1, M0, T1, 
#   , M0, T1, M1, T2, S2, 

ROUTE_DICT = {
    'U': 'up',
    'D': 'down',
    'L': 'left',
    'R': 'right',
}

DIRS = 'URDL'

SILENT = True

def test_route(route, silent=False):
    game = create_game(INIT_MAP)
    r = 0
    while not game.game_over:
        item = get_item(game)
        silent or print(f"T: {game.time}. H: [{game.hrow, game.hcol}. HP: {game.hero.health}. ", end='')
        if item is None:
            dir = ROUTE_DICT[route[r]]
            r = (r + 1) % len(route)
            silent or print(f"Nothing here. Hero moves {dir}")
            move_hero(game, dir)
        else:
            silent or print(f"{item.name} here. Explore")
            explore(game)
    return game

def build_routes(game, max_routes, route_length=10):
    routes = []
    for i in range(max_routes):
        route = ''
        for j in range(route_length):
            route += random.choice(DIRS)
        routes.append(route)
    return routes


In [107]:
SMALL_LOOP = 'URDL'
LEFT_COLS = 'RULRUUULDDDD'

g = test_route(LEFT_COLS, silent=True)
print_game(g)


Hero [(0, 1)] : 3 Level, 45 HP, 1005 gold, 905 glory
Time: 105. Hero alive. Game in progress. Game won
S2, t2, M1, T1, M0,   , 
T1, m0, S1, M1, S0, T0, 
M1, m0, M2, M2, M0, M0, 
T0, S0, M1, S1, M0, T1, 
  , M0, T1, M1, T2, S2, 


In [108]:
# rs = build_routes(game, 100)
for r in build_routes(game, 100, 10):
    game = test_route(r, True)
    if game.hero.gold >= game.win_gold:
        print(f"Route {r}")
        print_game(game, True)

Route RDDDRLLDDU
Hero [(4, 2)] : 2 Level, 90 HP, 1000 gold, 475 glory
Time: 111. Hero alive. Game in progress. Game won
